In [1]:
import exp_classification as exp

## Experiment 0: Baseline

In [3]:
exp.model_framework = 'Sklearn'
exp.seq_len = 1
exp.run_baseline()

----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                  9834.404751
Equity Peak [$]                   10615.64144
Return [%]                          -1.655952
Buy & Hold Return [%]               35.237428
Return (Ann.) [%]                   -0.841408
Volatility (Ann.) [%]                3.539551
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.476103
Avg. Drawdown [%]                    -1.12527
Max. Drawdown Duration      417 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                  233
Win Rate [%]                          47.2103
Best Trade [%]                       8.316799
Worst Trade [%]                    -11.

## Experiment 1: Logistic Regression

### AAPL

In [2]:
exp.model_framework = 'Sklearn'
exp.model_name = 'LogisticRegression'
exp.C = 10
exp.period = None
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [3]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 1
Train Ratio: 0.7
Test Ratio: 0.2
C: 10
Batch Size: 512
Input Size: 5
Hidden Size: 128
Number of Layers: 1
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 1000
Max Iterations: 1000
Model Name: LogisticRegression
Model Framework: Sklearn
----------------------------------------
Training LogisticRegression model...
(1792, 16) (1792,)
Train Acc: 0.5463, Val Acc: 0.5400
Test Acc: 0.5120, Test AUC: 0.5000
498 498
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
11

In [7]:
y_pred[26]

0

## Experiment 2: SVM

In [2]:
exp.model_framework = 'Sklearn'
exp.model_name = 'SVM'
exp.C = 0.1
exp.max_iter = 10000
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [38]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 1
Train Ratio: 0.7
Test Ratio: 0.2
Batch Size: 32
Input Size: 5
Hidden Size: 128
Number of Layers: 1
Number of Classes: 1
Learning Rate: 1e-05
Number of Epochs: 1000
Max Iterations: 10000
Model Name: SVM
Model Framework: Sklearn
----------------------------------------
Training SVM model...
(1792, 16) (1792,)
Train Acc: 0.5352, Val Acc: 0.5150
Test Acc: 0.5161, Test AUC: 0.5000
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                 10685.769975
Equity Peak [$]                  10685.769975
Return [%]                             6.8577
Buy & Hold Return [%]

## Experiment 3: LSTM

In [2]:
exp.model_framework = 'PyTorch'
exp.model_name = 'LSTMClassifier'
exp.seq_len = 10
exp.period = None
exp.n_epochs = 1000
exp.hidden_size = 64
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [3]:
y_pred = exp.run()

/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 5
Hidden Size: 64
Number of Layers: 1
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 1000
Max Iterations: 1000
Model Name: LSTMClassifier
Model Framework: PyTorch
----------------------------------------
Training LSTMClassifier model...
LSTMClassifier(
  (lstm): LSTM(16, 64, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.5428, Train Acc: 0.6753,             Val Acc: 0.5183, Val AUC: 0.5129, lr: 0.001000
Epoch: 200, Train Loss: 0.3120, Train Acc: 0.8458,             Val Acc: 0.5079, Val AUC: 0.5021, lr: 0.001000
Epoch: 300, Train Loss: 0.153

## Experiment 4: MLP

In [ ]:
exp.model_framework = 'Sklearn'
exp.model_name = 'MLP'
exp.seq_len = 1
exp.scale = True
exp.tol = 1e-6

In [ ]:
y_pred = exp.run()

Tiker: AAPL
Period: 10y
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 1
Train Ratio: 0.7
Test Ratio: 0.2
Batch Size: 32
Input Size: 16
Hidden Size: 128
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.0001
Number of Epochs: 1000
Model Name: MLP
Model Framework: Sklearn
----------------------------------------
Training MLP model...
(1792, 16) (1792,)
Iteration 1, loss = 0.69702402
Validation score: 0.500000
Iteration 2, loss = 0.69254541
Validation score: 0.511111
Iteration 3, loss = 0.69100649
Validation score: 0.494444
Iteration 4, loss = 0.68986667
Validation score: 0.505556
Iteration 5, loss = 0.68878750
Validation score: 0.527778
Iteration 6, loss = 0.68751922
Validation score: 0.527778
Iteration 7, loss = 0.68676401
Validation score: 0.511111
Iteration 8, loss = 0.68651601
Validation score: 0.538889
Iteration 9

## Experiment 5: AttLSTM

In [6]:
exp.model_framework = 'PyTorch'
exp.model_name = 'AttLSTMClassifier'
exp.seq_len = 10
exp.period = None
exp.n_epochs = 2000
exp.hidden_size = 128
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [7]:
y_pred = exp.run()

/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 5
Hidden Size: 128
Number of Layers: 1
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 2000
Max Iterations: 1000
Model Name: AttLSTMClassifier
Model Framework: PyTorch
----------------------------------------
Training AttLSTMClassifier model...
AttLSTMClassifier(
  (Tanh): Tanh()
  (lstm): LSTM(16, 128, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (batch_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.6440, Train Acc: 0.5766,             Val Acc: 0.4921, Val AUC: 0.4852, lr: 0.001000
Epoch: 200, Tra

## Experiment 6: GRU

In [5]:
exp.model_framework = 'PyTorch'
exp.model_name = 'GRUClassifier'
exp.seq_len = 10
exp.period = None
exp.n_epochs = 2000
exp.hidden_size = 128
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [6]:
y_pred = exp.run()

/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 5
Hidden Size: 128
Number of Layers: 1
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 2000
Max Iterations: 1000
Model Name: GRUClassifier
Model Framework: PyTorch
----------------------------------------
Training GRUClassifier model...
GRUClassifier(
  (gru): GRU(16, 128, batch_first=True, dropout=0.8)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.1462, Train Acc: 0.7443,             Val Acc: 0.4974, Val AUC: 0.4930, lr: 0.001000
Epoch: 200, Train Loss: 0.0065, Train Acc: 0.9400,             Val Acc: 0.4712, Val AUC: 0.4726, lr: 0.001000
Epoch: 300, Train Loss: 0.0010,